I am going to perform an exploratory data analysis (EDA) on all source tables to understand the data and identify any inconsistencies. Using SQL, I will combine these tables into a single summary table containing all necessary fields for the main analysis. I will clean and validate the data, handling around 10 million rows, perform checks to ensure accuracy, and explore the tables to understand patterns and relationships. Finally, I am going to create and save a cleaned, consolidated summary table to the database for future use.

In [61]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from urllib.parse import quote_plus

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="PAQ2026@",
    database="inventory"
)

user = "root"
password = "PAQ2026%40"
host = "localhost"
database = "inventory"

# Create SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}", echo=False)

# Query all tables
query = "SHOW TABLES;"
tables_df = pd.read_sql(query, engine)

tables_df


,Tables_in_inventory
0,begin_inventory
1,end_inventory
2,purchase_prices
3,purchases
4,sales
5,supplier_sales_analysis
6,vendor_invoice


### 1. Check Table Row Counts

In this step, we are querying the `information_schema.tables` to get an overview of all tables in the `inventory` database along with their respective row counts.  

This helps us understand **how much data each table contains** before we start the detailed analysis.  

- `table_name` → Name of the table  
- `table_rows` → Number of rows in the table  
- The result is sorted in descending order to easily identify the largest tables.


In [7]:

query = """
SELECT 
    table_name AS `Table`,
    table_rows AS `Row_Count`
FROM 
    information_schema.tables
WHERE 
    table_schema = 'inventory'
ORDER BY 
    table_rows DESC;
"""

counts_df = pd.read_sql(query, engine)


counts_df


,Table,Row_Count
0,sales,11309036
1,purchases,2280728
2,end_inventory,222351
3,begin_inventory,204513
4,supplier_sales_analysis,52825
5,purchase_prices,12257
6,vendor_invoice,5445


### 2. Preview First 5 Rows of Each Table

Here, we loop through all tables in the `inventory` database and display the **first 5 rows** of each.  

Purpose of this step:  
- Understand the **structure and sample data** of each table.  
- Quickly check **column names, data types, and example values**.  
- Helps in planning further **analysis and transformations**.

- `table_list` → List of table names obtained from the previous row-count query.  
- `pd.read_sql(...LIMIT 5)` → Retrieves a small sample for inspection.  
- `display(df)` → Ensures the table is nicely formatted in Jupyter.


In [8]:


table_list = counts_df['Table'].tolist()  

for table in table_list:
    print(f"\nFirst 5 rows of table: {table}")
    df = pd.read_sql(f"SELECT * FROM `{table}` LIMIT 5;", engine)
    display(df)   



First 5 rows of table: sales


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY



First 5 rows of table: purchases


,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26,1
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.4,1
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.1,1
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,2024-01-09,2024-02-16,21.32,5,106.6,1



First 5 rows of table: end_inventory


...

,VendorNumber,VendorName,Brand,PurchasePrice,Description,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars,TotalFreight,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,GrossProfit,ProfitMarginPercent,StockTurnOver,InventoryToSalesRatio,ExciseTaxPercent
0,2,"IRA GOLDMAN AND WILLIAMS, LLP",90085,23.86,Ch Lilian 09 Ladouys St Este,36.99,8.0,190.88,27.08,18.0,665.82,295.92,2.00,474.94,71.33,2.25,0.44,0.30
1,2,"IRA GOLDMAN AND WILLIAMS, LLP",90609,17.00,Flavor Essence Variety 5 Pak,24.99,320.0,5440.00,27.08,24.0,599.76,449.82,0.52,-4840.24,-807.03,0.08,13.33,0.09
2,54,AAPER ALCOHOL & CHEMICAL CO,990,105.07,Ethyl Alcohol 200 Proof,134.49,1.0,105.07,0.48,0.0,0.00,0.00,0.00,-105.07,0.00,0.00,0.00,0.00
3,60,ADAMBA IMPORTS INTL INC,771,11.44,Bak's Krupnik Honey Liqueur,14.99,39.0,446.16,367.52,47.0,704.53,494.67,37.01,258.37,36.67,1.21,0.83,5.25
4,60,ADAMBA IMPORTS INTL INC,3401,11.10,Vesica Vodka,14.99,6.0,66.60,367.52,0.0,0.00,0.00,0.00,-66.60,0.00,0.00,0.00,0.00



First 5 rows of table: purchase_prices


,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.



First 5 rows of table: vendor_invoice


,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.6,4.61,
3,480,BACARDI USA INC,2024-01-12,8106,2023-12-20,2024-02-05,10100,137483.78,2935.2,
4,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.2,


In [9]:
purchases = pd.read_sql("select * from purchases;",engine)
purchases.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'VendorNumber',
       'VendorName', 'PONumber', 'PODate', 'ReceivingDate', 'InvoiceDate',
       'PayDate', 'PurchasePrice', 'Quantity', 'Dollars', 'Classification'],
      dtype='object')

### 3. Inspect Columns of `vendor_invoice` Table

In this step, we query the `INFORMATION_SCHEMA.COLUMNS` to get a **list of all column names** in the `vendor_invoice` table.  

Purpose of this step:  
- Understand the **structure of the table** before performing any analysis.  
- Helps identify **relevant columns** for supplier and inventory analysis.  
- Ensures we know the **exact column names** to avoid errors in queries or data processing.


In [10]:
pd.read_sql("""
SELECT COLUMN_NAME 
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE TABLE_SCHEMA = 'inventory' AND TABLE_NAME = 'vendor_invoice';
""", engine)


,COLUMN_NAME
0,Approval
1,Dollars
2,Freight
3,InvoiceDate
4,PayDate
5,PODate
6,PONumber
7,Quantity
8,VendorName
9,VendorNumber


In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

### 4. Calculate Total Freight Cost per Vendor

In this step, we aggregate the freight costs from the `vendor_invoice` table to understand **how much freight each vendor is charging**.  

Purpose of this step:  
- Identify vendors with **high shipping costs**.  
- Helps in **cost optimization** by analyzing freight impact on overall procurement.  
- `VendorNumber` and `VendorName` → Used to group the data.  
- `SUM(Freight)` → Computes the total freight cost per vendor.


In [42]:
freight = """
SELECT VendorNumber, VendorName, SUM(Freight) AS total_freight
FROM vendor_invoice
GROUP BY VendorNumber, VendorName;
"""

freight_summary = pd.read_sql(freight, engine)
freight_summary


,VendorNumber,VendorName,total_freight
0,105,ALTAMAR BRANDS LLC,62.39
1,4466,AMERICAN VINTAGE BEVERAGE,793.91
2,388,ATLANTIC IMPORTING COMPANY,211.74
3,480,BACARDI USA INC,89286.27
4,516,BANFI PRODUCTS CORP,8510.41
...,...,...,...
124,4901,LAUREATE IMPORTS CO,0.72
125,90059,BLACK COVE BEVERAGES,74.84
126,5083,LOYAL DOG WINERY,10.68
127,1587,VINEYARD BRANDS LLC,54.23


In [13]:
print("\n" + "="*50)
print("STEP 3: CREATING MASTER ANALYSIS TABLE")
print("="*50)


STEP 3: CREATING MASTER ANALYSIS TABLE


### 5. Consolidated Vendor & Brand Performance Analysis

In this step, we create a **comprehensive view** combining purchase, freight, and sales data at the vendor and brand level.  

Key steps performed in this query:  
1. **`freight_agg` CTE** → Aggregates total freight cost per vendor from `vendor_invoice`.  
2. **`purchase_agg` CTE** → Aggregates purchase quantity and dollars per vendor and brand. It also joins with `purchase_prices` to get the actual price.  
3. **`sales_agg` CTE** → Aggregates sales quantity, sales dollars, total price, and excise tax per vendor and brand.  
4. **Final SELECT** → Joins all three CTEs to get a **single dataframe** containing:  
   - Vendor information (`VendorNumber`, `VendorName`)  
   - Brand and purchase details (`Brand`, `PurchasePrice`, `Description`, `ActualPrice`, `TotalPurchaseQuantity`, `TotalPurchaseDollars`)  
   - Freight (`total_freight`)  
   - Sales details (`TotalSalesQuantity`, `TotalSalesDollars`, `TotalSalesPrice`, `TotalExciseTax`)  
5. **ORDER BY `TotalPurchaseDollars` DESC** → Highlights vendors and brands with the highest purchase spend.  

Purpose:  
- Provides a **holistic view of supplier performance** combining cost (purchase + freight) and revenue (sales).  
- Enables **identification of high-spend vendors**, high-margin brands, and cost optimization opportunities.


In [44]:

final_query = """
WITH freight_agg AS (
    SELECT 
        VendorNumber, 
        VendorName, 
        SUM(Freight) AS total_freight
    FROM vendor_invoice
    GROUP BY VendorNumber, VendorName
),
purchase_agg AS (
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.PurchasePrice,
        p.Description,
        pp.Price AS ActualPrice,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
        ON p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.PurchasePrice,
        p.Description,
        pp.Price
),
sales_agg AS (
    SELECT 
        VendorNo,
        Brand,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(SalesPrice) AS TotalSalesPrice,
        SUM(ExciseTax) AS TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
)
SELECT 
    p.VendorNumber,
    p.VendorName,
    p.Brand,
    p.PurchasePrice,
    p.Description,
    p.ActualPrice,
    p.TotalPurchaseQuantity,
    p.TotalPurchaseDollars,
    f.total_freight,
    s.TotalSalesQuantity,
    s.TotalSalesDollars,
    s.TotalSalesPrice,
    s.TotalExciseTax
FROM purchase_agg p
LEFT JOIN freight_agg f
    ON p.VendorNumber = f.VendorNumber
    AND p.VendorName = f.VendorName
LEFT JOIN sales_agg s
    ON p.VendorNumber = s.VendorNo
    AND p.Brand = s.Brand
ORDER BY p.TotalPurchaseDollars DESC;
"""

# 3️⃣ Execute the query and load into Pandas
df_final = pd.read_sql(final_query, engine)

# 4️⃣ Preview the first rows
df_final


,VendorNumber,VendorName,Brand,PurchasePrice,Description,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars,total_freight,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax
0,1128,BROWN-FORMAN CORP,1233,26.27,Jack Daniels No 7 Black,36.99,145080.0,3811251.60,68601.68,142049.0,5.101920e+06,672819.31,260999.20
1,4425,MARTIGNETTI COMPANIES,3405,23.19,Tito's Handmade Vodka,28.99,164038.0,3804041.22,144719.92,160247.0,4.819073e+06,561512.37,294438.66
2,17035,PERNOD RICARD USA,8068,18.24,Absolut 80 Proof,24.99,187407.0,3418303.68,123780.22,187140.0,4.538121e+06,461140.15,343854.07
3,3960,DIAGEO NORTH AMERICA INC,4261,16.17,Capt Morgan Spiced Rum,22.99,201682.0,3261197.94,257032.07,200412.0,4.475973e+06,420050.01,368242.80
4,3960,DIAGEO NORTH AMERICA INC,3545,21.89,Ketel One Vodka,29.99,138109.0,3023206.01,257032.07,135838.0,4.223108e+06,545778.28,249587.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,1.32,Concannon Glen Ellen Wh Zin,4.99,2.0,2.64,27100.41,5.0,1.595000e+01,10.96,0.55
10688,8004,SAZERAC CO INC,5683,0.39,Dr McGillicuddy's Apple Pie,0.49,6.0,2.34,50293.62,134.0,6.566000e+01,1.47,7.04
10689,3924,HEAVEN HILL DISTILLERIES,9123,0.74,Deep Eddy Vodka,0.99,2.0,1.48,14069.87,2.0,1.980000e+00,0.99,0.10
10690,3960,DIAGEO NORTH AMERICA INC,6127,1.47,The Club Strawbry Margarita,1.99,1.0,1.47,257032.07,72.0,1.432800e+02,77.61,15.12


### 6. Check Data Types of Consolidated Data

In this step, we inspect the **data types** of each column in `df_final` to ensure the data is in the correct format for analysis.  

Purpose of this step:  
- Identify **numeric columns** that can be used for calculations (e.g., `TotalPurchaseDollars`, `TotalSalesDollars`).  
- Identify **categorical columns** suitable for grouping or aggregation (e.g., `VendorName`, `Brand`).  
- Detect potential **data type issues** that might affect visualizations or further analysis.


In [35]:
df_final.dtypes

VendorNumber              object
VendorName                object
Brand                     object
PurchasePrice             object
Description               object
ActualPrice               object
TotalPurchaseQuantity    float64
TotalPurchaseDollars     float64
TotalFreight             float64
TotalSalesQuantity       float64
TotalSalesDollars        float64
TotalSalesPrice          float64
TotalExciseTax           float64
GrossProfit              float64
ProfitMarginPercent      float64
InventoryToSalesRatio    float64
ExciseTaxPercent         float64
SalesToPurchaseRatio     float64
dtype: object

### 7. Check for Missing Values

In this step, we examine `df_final` for any **missing or null values** in each column.  

Purpose of this step:  
- Identify columns that may require **data cleaning or imputation**.  
- Understand **completeness of the data**, especially important for calculations involving purchase, sales, or freight.  
- Helps prevent **errors in downstream analysis or visualizations** due to missing values.


In [45]:
df_final.isnull().sum()

VendorNumber               0
VendorName                 0
Brand                      0
PurchasePrice              0
Description                0
ActualPrice                0
TotalPurchaseQuantity      0
TotalPurchaseDollars       0
total_freight              0
TotalSalesQuantity       178
TotalSalesDollars        178
TotalSalesPrice          178
TotalExciseTax           178
dtype: int64

### 8. Inspect Unique Vendors

In this step, we extract the **unique vendor names** from `df_final`.  

Purpose of this step:  
- Understand **how many distinct vendors** are present in the dataset.  
- Helps in grouping, filtering, or analyzing vendor-specific metrics in subsequent analysis.  
- Ensures we are aware of all vendors before performing **vendor-level performance comparisons**.


In [46]:
df_final['VendorName'].unique()

array(['BROWN-FORMAN CORP          ', 'MARTIGNETTI COMPANIES',
       'PERNOD RICARD USA          ', 'DIAGEO NORTH AMERICA INC   ',
       'BACARDI USA INC            ', 'JIM BEAM BRANDS COMPANY    ',
       'MAJESTIC FINE WINES        ', 'ULTRA BEVERAGE COMPANY LLP ',
       'STOLI GROUP,(USA) LLC      ', 'PROXIMO SPIRITS INC.       ',
       'MOET HENNESSY USA INC      ', 'CAMPARI AMERICA            ',
       'SAZERAC CO INC             ', 'CONSTELLATION BRANDS INC   ',
       'M S WALKER INC             ', 'SAZERAC NORTH AMERICA INC. ',
       'PALM BAY INTERNATIONAL INC ', 'REMY COINTREAU USA INC     ',
       'SIDNEY FRANK IMPORTING CO  ', 'E & J GALLO WINERY         ',
       'WILLIAM GRANT & SONS INC   ', 'HEAVEN HILL DISTILLERIES   ',
       'DISARONNO INTERNATIONAL LLC', 'EDRINGTON AMERICAS         ',
       'CASTLE BRANDS CORP.        ', 'SOUTHERN WINE & SPIRITS NE ',
       'STE MICHELLE WINE ESTATES  ', 'TRINCHERO FAMILY ESTATES   ',
       'MHW LTD                    ', 'W

### 9. Clean Vendor Names

In this step, we remove **leading and trailing spaces** from the `VendorName` column using the `str.strip()` method.  

Purpose of this step:  
- Ensures **consistency in vendor names**, avoiding duplicates caused by extra spaces.  
- Important for **accurate grouping, aggregation, and comparison** in vendor-level analysis.  
- Prevents errors in joins or merges with other tables that use `VendorName`.


In [47]:
df_final['VendorName']=df_final['VendorName'].str.strip()

### 10. Inspect Unique Product Descriptions

In this step, we extract the **unique values** from the `Description` column.  

Purpose of this step:  
- Understand **what products or items are present** in the dataset.  
- Helps identify **distinct SKUs or product types** for vendor and inventory analysis.  
- Useful for detecting **data inconsistencies or duplicates** in product naming.


In [48]:
df_final['Description'].unique()

array(['Jack Daniels No 7 Black', "Tito's Handmade Vodka",
       'Absolut 80 Proof', ..., 'Crown Royal Apple',
       'Concannon Glen Ellen Wh Zin', 'The Club Strawbry Margarita'],
      shape=(9651,), dtype=object)

### 11. Handle Missing Values

In this step, we fill all missing (`NaN`) values in `df_final` with `0` using `fillna(0)`.  

Purpose of this step:  
- Ensures that **calculations and aggregations** on numeric columns do not fail due to missing values.  
- Makes the dataset **complete and ready for analysis**.  
- Important for metrics like **TotalPurchaseDollars, TotalSalesDollars, and total_freight**, where missing values could distort results.


In [49]:
df_final.fillna(0,inplace=True)

### 12. Calculate Gross Profit

In this step, we create a new column `GrossProfit` by subtracting `TotalPurchaseDollars` from `TotalSalesDollars`.  

Purpose of this step:  
- Measures the **profit generated** by each vendor and brand before accounting for other expenses.  
- Helps identify **high-margin vendors or products**.  
- Provides a key metric for **supplier performance and profitability analysis**.


In [50]:
import numpy as np

df_final['GrossProfit'] = df_final['TotalSalesDollars'] - df_final['TotalPurchaseDollars']


### 13. Identify Minimum Gross Profit

In this step, we calculate the **minimum value of `GrossProfit`** across all vendors and brands using `df_final['GrossProfit'].min()`.  

Purpose of this step:  
- Identify vendors or products with **negative or very low profitability**.  
- Helps flag **underperforming vendors or loss-making products**.  
- Provides insight for **cost optimization or supplier negotiation**.


In [51]:
df_final['GrossProfit'].min()

np.float64(-52002.78000000001)

### 14. Calculate Profit Margin Percentage

In this step, we create a new column `ProfitMargin%` by dividing `GrossProfit` by `TotalSalesDollars` and multiplying by 100.  

Purpose of this step:  
- Measures the **profitability relative to sales** for each vendor and brand.  
- Helps identify **high-margin vs low-margin vendors or products**.  
- Useful for **comparing performance across vendors** regardless of absolute sales volume.


In [52]:
df_final['ProfitMargin%'] = (df_final['GrossProfit']/df_final['TotalSalesDollars'])*100

### 15. Calculate Inventory-to-Sales Ratio

In this step, we create a new column `InventoryToSalesRatio` by dividing `TotalPurchaseQuantity` by `TotalSalesQuantity`.  

Purpose of this step:  
- Measures the **amount of inventory relative to sales** for each vendor or product.  
- Helps identify **overstocked or slow-moving items** (high ratio) and **fast-moving items** (low ratio).  
- Replacing 0 with `NaN` in `TotalSalesQuantity` prevents **division by zero errors**.


In [53]:
df_final['InventoryToSalesRatio'] = df_final['TotalPurchaseQuantity'] / df_final['TotalSalesQuantity'].replace(0, np.nan)


### 16. Identify Loss-Making Products

In this step, we filter `df_final` to find all rows where `GrossProfit` is **less than 0** and sort them in ascending order of `GrossProfit`.  

Purpose of this step:  
- Highlight **products or vendors generating losses**.  
- Helps in **supplier negotiation, inventory reduction, or discontinuation decisions**.  
- Sorting by `GrossProfit` makes it easy to **see the most loss-making items first**.
``


In [54]:
loss_products = df_final[df_final['GrossProfit'] < 0].sort_values(by='GrossProfit')


### 17. Calculate Excise Tax Percentage

In this step, we create a new column `ExciseTaxPercent` by dividing `TotalExciseTax` by `TotalSalesDollars` and multiplying by 100.  

Purpose of this step:  
- Measures the **tax burden relative to sales** for each vendor or product.  
- Helps in understanding **how excise tax impacts profitability**.  
- Filling `NaN` in `TotalExciseTax` with 0 and replacing 0 in `TotalSalesDollars` with `NaN` ensures **no division by zero er**


In [55]:
df_final['ExciseTaxPercent'] = df_final['TotalExciseTax'].fillna(0) / df_final['TotalSalesDollars'].replace(0, np.nan) * 100


### 18. Calculate Sales-to-Purchase Ratio

In this step, we create a new column `SalesToPurchaseRatio` by dividing `TotalSalesDollars` by `TotalPurchaseDollars`.  

Purpose of this step:  
- Measures the **efficiency of converting purchases into sales** for each vendor or product.  
- A **ratio > 1** indicates that sales exceed purchases (profitable turnover), while **< 1** indicates p


In [56]:
df_final['SalesToPurchaseRatio']=df_final['TotalSalesDollars']/df_final['TotalPurchaseDollars']


### 19. Create Database Cursor

In this step, we create a **cursor object** from the database connection `conn`.  

Purpose of this step:  
- The cursor allows us to **execute SQL queries** and fetch results from the database.  
- Essential for **interacting with the database** beyond high-level methods like `pd.read_sql()`.  
- Provides control over **transaction management and query execution**.


In [62]:
cursor = conn.cursor()

### 20. Create `supplier_sales_analysis` Table in MySQL

In this step, we create a new table in the MySQL database to store the consolidated vendor and brand-level analysis.  

Key points:  
- Uses `CREATE TABLE IF NOT EXISTS` to **ensure the table is created only if it doesn’t already exist**.  
- Defines **columns and data types** for all key metrics including purchase, sales, freight, gross profit, profit margin, and inventory ratios.  
- Sets a **composite primary key** on `VendorNumber` and `Brand` to uniquely identify each record.  
- Commits the transaction and closes the connection after creation.  

Purpose:  
- Enables **storing the processed and enriched dataset** in a structured table for future queries, reporting, or dashboarding.  
- Makes it easy to **access vendor and brand performance metrics** directly from MySQL for BI tools like Power BI or


In [63]:

cursor = conn.cursor()


cursor.execute("""
CREATE TABLE IF NOT EXISTS supplier_sales_analysis (
    VendorNumber BIGINT,
    VendorName VARCHAR(100),
    Brand BIGINT,
    PurchasePrice DECIMAL(15,2),
    Description VARCHAR(100),
    ActualPrice DECIMAL(15,2), 
    TotalPurchaseQuantity DECIMAL(15,2), 
    TotalPurchaseDollars DECIMAL(15,2),
    TotalFreight DECIMAL(15,2),
    TotalSalesQuantity DECIMAL(15,2),
    TotalSalesDollars DECIMAL(15,2),
    TotalSalesPrice DECIMAL(15,2),
    TotalExciseTax DECIMAL(15,2), 
    GrossProfit DECIMAL(15,2), 
    ProfitMarginPercent DECIMAL(15,2),
    StockTurnOver DECIMAL(15,2),
    TotalFreight DECIMAL(15,2),
    InventoryToSalesRatio DECIMAL(15,2),
    ExciseTaxPercent DECIMAL(15,2),
    PRIMARY KEY (VendorNumber, Brand)
);
""")

conn.commit()
cursor.close()
conn.close()

print("✅ Table created successfully in MySQL!")


✅ Table created successfully in MySQL!


### 21. Rename Columns for Consistency

In this step, we rename the column `total_freight` to `TotalFreight` in `df_final`.  

Purpose of this step:  
- Ensures **consistent column naming** across the dataframe and MySQL table.  
- Prevents **mismatches or errors** when inserting data into the database or performing further analysis.  
- Helps maintain **readability and clarity** for future users of the notebook.


In [64]:
df_final = df_final.rename(columns={
    "total_freight": "TotalFreight"
})

### 22. Standardize Profit Margin Column Name

In this step, we rename the column `ProfitMargin%` to `ProfitMarginPercent` in `df_final`.  

Purpose of this step:  
- Ensures **consistent and database-friendly column names**.  
- Prevents issues when **inserting into MySQL**, as `%` is not ideal in column names.  
- Improves **readability and clarity** for analysis and reporting.


In [65]:
df_final = df_final.rename(columns={
    "ProfitMargin%": "ProfitMarginPercent"
})

### 23. Define Data Cleaning Function for MySQL Insertion

In this step, we define a function `clean_dataframe_for_mysql` to **prepare the dataframe for database insertion**.  

Purpose of this function:  
- Ensures **numeric columns are properly formatted** for MySQL `DECIMAL` types.  
- Replaces **infinite values** (`inf`, `-inf`) with `NaN`.  
- Fills **NaN values with 0** to avoid insertion errors.  
- Rounds numeric columns to a fixed number of **decimal places** to match database precision.  

Parameters:  
- `df` → The dataframe to clean  
- `decimal_cols` → List of columns to format as decimals  
- `decimals` → Number of decimal places (default is 2)


In [66]:
import pandas as pd
import numpy as np

def clean_dataframe_for_mysql(df, decimal_cols, decimals=2):
    """
    Cleans a DataFrame for MySQL insertion:
      - Converts to numeric where needed
      - Replaces inf/-inf and 'inf' strings with NaN
      - Fills NaN with 0
      - Rounds to fixed decimal places
    """
    df = df.copy()

    for col in decimal_cols:
        if col in df.columns:
            # Replace any kind of inf
            df[col] = df[col].replace([np.inf, -np.inf, "inf", "Inf", "INF", "-inf", "-Inf", "-INF"], np.nan)
            
            # Convert to numeric
            df[col] = pd.to_numeric(df[col], errors="coerce")
            
            # Replace NaN with 0
            df[col] = df[col].fillna(0)
            
            # Round to match DECIMAL precision
            df[col] = df[col].round(decimals)
    
    return df


### 24. Apply Data Cleaning Function to Consolidated Data

In this step, we apply the `clean_dataframe_for_mysql` function to `df_final` for the specified `decimal_cols`.  

Purpose of this step:  
- Ensures all **numeric columns are cleaned, NaN values replaced, and rounded** for MySQL compatibility.  
- Produces a **ready-to-insert dataframe** called `df_final_clean`.  
- Prevents **data type or precision errors** when loading into the `supplier_sales_analysis` table.


In [67]:
decimal_cols = [
    "PurchasePrice", "ActualPrice", "TotalPurchaseQuantity", "TotalPurchaseDollars",
    "TotalFreight", "TotalSalesQuantity", "TotalSalesDollars", "TotalSalesPrice",
    "TotalExciseTax", "GrossProfit", "ProfitMarginPercent", "StockTurnOver",
    "InventoryToSalesRatio", "ExciseTaxPercent"
]

df_final_clean = clean_dataframe_for_mysql(df_final, decimal_cols, decimals=2)


### 25. Insert Cleaned Data into MySQL Table

In this step, we insert `df_final_clean` into the MySQL table `supplier_sales_analysis` using the `to_sql` method.  

Purpose of this step:  
- Loads the **processed and cleaned vendor & brand performance data** into the database.  
- Uses `if_exists="append"` to **add new records** without overwriting existing data.  
- `index=False` ensures the dataframe index is **not inserted as a separate column**.  
- Makes the data **accessible for reporting, BI tools, or further SQL queries**.


In [68]:
df_final_clean.to_sql(
    "supplier_sales_analysis",
    con=engine,
    if_exists="append",
    index=False
)


ProgrammingError: (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'SalesToPurchaseRatio' in 'field list'
[SQL: INSERT INTO supplier_sales_analysis (`VendorNumber`, `VendorName`, `Brand`, `PurchasePrice`, `Description`, `ActualPrice`, `TotalPurchaseQuantity`, `TotalPurchaseDollars`, `TotalFreight`, `TotalSalesQuantity`, `TotalSalesDollars`, `TotalSalesPrice`, `TotalExciseTax`, `GrossProfit`, `ProfitMarginPercent`, `InventoryToSalesRatio`, `ExciseTaxPercent`, `SalesToPurchaseRatio`) VALUES (%(VendorNumber)s, %(VendorName)s, %(Brand)s, %(PurchasePrice)s, %(Description)s, %(ActualPrice)s, %(TotalPurchaseQuantity)s, %(TotalPurchaseDollars)s, %(TotalFreight)s, %(TotalSalesQuantity)s, %(TotalSalesDollars)s, %(TotalSalesPrice)s, %(TotalExciseTax)s, %(GrossProfit)s, %(ProfitMarginPercent)s, %(InventoryToSalesRatio)s, %(ExciseTaxPercent)s, %(SalesToPurchaseRatio)s)]
[parameters: [{'VendorNumber': '1128', 'VendorName': 'BROWN-FORMAN CORP', 'Brand': '1233', 'PurchasePrice': 

### 26. Verify Data Insertion in MySQL

In this step, we query the `supplier_sales_analysis` table to **check the first 5 rows**, ordered by `TotalSalesDollars` in descending order.  

Purpose of this step:  
- Confirms that the **data was successfully inserted** into the MySQL table.  
- Quickly inspects the **top-performing vendors/brands by sales**.  
- Acts as a **validation step** before


In [ ]:
df_check = pd.read_sql("SELECT * FROM supplier_sales_analysis order by TotalSalesDollars desc LIMIT 5;", con=engine)
print(df_check)
